In [1]:
import pandas as pd
import os
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from models.create_dataset import create_dataset


In [2]:

df = create_dataset()

invalid cols ['NO_Solar', 'NO_Solar_lag_1', 'NO_Solar_lag_2', 'NO_Solar_lag_3', 'NO_Solar_lag_7']
68809
68641


In [3]:
target_col = 'DK1_price'

In [7]:
y = df[target_col]
X = df.drop(target_col, axis=1)
print(X.index[0])
# Pivot hourly index out to columns so index is only date
pivot_columns = [col for col in X.columns if not col.startswith('day_of_week')]
X = X.pivot_table(index=X.index.date, columns=X.index.hour, values=pivot_columns)
X = X.dropna()
print(X.shape[0])
X = X.loc[:, (X != 0).any(axis=0)]
print(X.shape[0])
# and some are 0 almost always, drop features with a MAD below threshold
X = X.loc[:, X.sub(X.median(axis=0), axis=1).abs().median(axis=0) > 0.01]
print(X.shape[0])
X.index = pd.to_datetime(X.index)
X['day_of_week'] = X.index.dayofweek
print(X.shape[0])
# to dummies
# day_of_week_0 column when day_of_week is 0, i.e. monday. 1 if monday, 0 otherwise
X['day_of_week_0'] = X['day_of_week'].apply(lambda x: 1 if x == 0 else 0)
X = pd.get_dummies(X, columns=['day_of_week'], drop_first=True) # last one should not be there, but we still use it?

2015-03-03 23:00:00
2860
2860
2860
2860


In [6]:
X.index[0]

Timestamp('2015-03-04 00:00:00')

In [22]:
# plot march
march_df = dk_df.loc['2017-03-10':'2017-03-21']
fig = px.line(march_df, x=march_df.index, y=march_df.columns)
# add MW as y axis0
fig.update_yaxes(title_text='MW')
# update x
fig.update_xaxes(title_text='Date')
# highlight the 17th
fig.add_vline(x='2017-03-17', line_width=3, line_dash="dash", line_color="green")
fig.add_vline(x='2017-03-18', line_width=3, line_dash="dash", line_color="green")
# add title
fig.update_layout(title_text='March 2017 Wind Power Production in Denmark')
# add legend stating interpolation period
fig.add_annotation(x='2017-03-17', y=0.5, text="interpolated data", showarrow=True)

# save fig png
# path =
os.chdir('..')
os.chdir('plots')
fig.write_image("march.png")